In [2]:
import os
os.getcwd()

'/home/jovyan/ArditArifi'

In [3]:
source_path="/home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6"
output_path="/home/jovyan/student-storages/GROUP3/ArditArifi/output"

! mkdir -p output_path

Analysis

In [4]:
import glob
# ANALYSIS
import xarray as xr
import pandas as p
import numpy as np
# PLOTTING
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.colors as colors


In [5]:

# REGIONS
region_label = ["ARC", "ANT"]
arctic_lat = [60, 90]
antarctic_lat = [-90, -60]

# SEASONS
seasons_label = ["DJF", "MAM", "JJA", "SON"]
seasons = {
    "DJF": [12, 1, 2],   # Dec, Jan, Feb
    "MAM": [3, 4, 5],    # Mar, Apr, May
    "JJA": [6, 7, 8],    # Jun, Jul, Aug
    "SON": [9, 10, 11],  # Sep, Oct, Nov
}

# Create combined region-season labels
region_season_label = [f"{region} {season}" for region in region_label for season in seasons_label]

# EXPERIMENTS
experiments = ["2xss", "2xdust", "2xfire", "2xDMS"]

# MODELS
models = ["IPSL-CM6A-LR-INCA", "UKESM1-0-LL", "NorESM2-LM"]

# PATTERN FUNCTION
patter = lambda model, var, exp: f"{source_path}/{var}*{model}*piClim-{exp}*.nc"

In [6]:
# CONSTRUCT TABLE
mean_values = np.full(
    shape=(len(region_season_label), len(experiments), len(models)),
    fill_value=np.nan
)
std_values = np.full(
    shape=(len(region_season_label), len(experiments), len(models)),
    fill_value=np.nan
)


# LOAD DATA
model = ""
var = "rtmt"

# Calculate total steps for progress
total_steps = len(models) * len(experiments) * len(seasons_label)
current_step = 0
         

# MAIN LOOP
for idx_model, model in enumerate(models):
    try:
        # Open control dataset for this model
        ds_ctl = xr.open_mfdataset(patter(model, var, "control"), combine="by_coords")
    except Exception as e:
        print(f"Error opening control files {patter(model, var, 'control')}: {e}")
        # Skip all seasons for this model if control is missing
        current_step += len(seasons_label) * len(experiments)
        continue
        
    for idx_exp, exp in enumerate(experiments):
        # Try opening the experiment dataset
        try:
            ds_exp = xr.open_mfdataset(patter(model, var, exp), combine="by_coords")
        except Exception as e:
            print(f"Error opening experiment files {patter(model, var, exp)}: {e}")
            current_step += len(seasons_label)
            continue

        # Compute bias as experiment minus control
        ds_bias = ds_exp - ds_ctl

        for idx_season, season in enumerate(seasons_label):
            # Increment progress
            current_step += 1
            pct = (current_step / total_steps) * 100
            print(f"Progress: {current_step}/{total_steps} -> {pct:.2f}%")

            # 1) Select months for this season
            ds_season = ds_bias.sel(time=ds_bias.time.dt.month.isin(seasons[season]))

            # ---------------------
            # ARCTIC
            # ---------------------
            ds_season_arctic = ds_season.sel(
                lat=slice(arctic_lat[0], arctic_lat[1]),
                lon=slice(-180, 180)
            )
            # First, take time mean (removes the 'time' dimension)
            ds_arctic_time_mean = ds_season_arctic[var].mean(dim="time")

            # Define weights after subsetting region
            weights_arctic = np.cos(np.deg2rad(ds_arctic_time_mean.lat))

            # Now do the weighted mean/std over lat/lon
            arc_mean = ds_arctic_time_mean.weighted(weights_arctic).mean(dim=("lat","lon")).values
            arc_std  = ds_arctic_time_mean.weighted(weights_arctic).std(dim=("lat","lon")).values

            mean_values[idx_season, idx_exp, idx_model] = arc_mean
            std_values[idx_season, idx_exp, idx_model]  = arc_std
            
            # ---------------------
            # ANTARCTIC
            # ---------------------
            ds_season_antarctic = ds_season.sel(
                lat=slice(antarctic_lat[0], antarctic_lat[1]),
                lon=slice(-180, 180)
            )
            # Time mean first
            ds_antarctic_time_mean = ds_season_antarctic[var].mean(dim="time")

            weights_antarctic = np.cos(np.deg2rad(ds_antarctic_time_mean.lat))

            ant_mean = ds_antarctic_time_mean.weighted(weights_antarctic).mean(dim=("lat","lon")).values
            ant_std  = ds_antarctic_time_mean.weighted(weights_antarctic).std(dim=("lat","lon")).values

            mean_values[idx_season + 4, idx_exp, idx_model] = ant_mean
            std_values[idx_season + 4, idx_exp, idx_model]  = ant_std
            
# Finally, display the populated array
print("Shape of mean_values:", mean_values.shape)
print(mean_values)

Progress: 1/48 -> 2.08%
Progress: 2/48 -> 4.17%
Progress: 3/48 -> 6.25%
Progress: 4/48 -> 8.33%
Progress: 5/48 -> 10.42%
Progress: 6/48 -> 12.50%
Progress: 7/48 -> 14.58%
Progress: 8/48 -> 16.67%
Error opening files /home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6/rtmt*IPSL-CM6A-LR-INCA*piClim-2xfire*.nc: no files to open
Error opening files /home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6/rtmt*IPSL-CM6A-LR-INCA*piClim-2xDMS*.nc: no files to open
Progress: 17/48 -> 35.42%
Progress: 18/48 -> 37.50%
Progress: 19/48 -> 39.58%
Progress: 20/48 -> 41.67%
Progress: 21/48 -> 43.75%
Progress: 22/48 -> 45.83%
Progress: 23/48 -> 47.92%
Progress: 24/48 -> 50.00%
Progress: 25/48 -> 52.08%
Progress: 26/48 -> 54.17%
Progress: 27/48 -> 56.25%
Progress: 28/48 -> 58.33%
Progress: 29/48 -> 60.42%
Progress: 30/48 -> 62.50%
Progress: 31/48 -> 64.58%
Progress: 32/48 -> 66.67%
Progress: 33/48 -> 68.75%
Progress: 34/48 -> 70.83%
Progress: 35/48 -> 72.92%
Progress: 36/48 -> 75.00%
Progress: 37/48 -> 77.08%
P

In [8]:
import pandas as pd

bias = mean_values
for idx_exp in range(len(experiments)):
    # This will skip the control experiment at index 0
    exp_name = experiments[idx_exp]
    print("Experiment:", exp_name)
    
    # bias[:, idx_exp, :].shape -> (len(region_season_label), len(models))
    # bias[:, idx_exp, :].transpose().shape -> (len(models), len(region_season_label))
    # We want models as rows and region_season_label as columns
    df = pd.DataFrame(
        data=bias[:, idx_exp, :].transpose(),
        index=models,
        columns=region_season_label
    )

    print(df)
    print()  # Blank line for readability

Experiment: 2xss
                    ARC DJF   ARC MAM   ARC JJA   ARC SON   ANT DJF   ANT MAM  \
IPSL-CM6A-LR-INCA  0.157356 -0.600881 -1.143472 -0.077902 -0.637405 -0.164734   
UKESM1-0-LL       -0.260221  0.168703 -0.497273 -0.050565 -0.399657 -0.295128   
NorESM2-LM        -0.802078 -0.343978 -0.872966 -0.441364 -1.438295 -0.408204   

                    ANT JJA   ANT SON  
IPSL-CM6A-LR-INCA -0.064649 -0.080119  
UKESM1-0-LL        0.103706 -0.346260  
NorESM2-LM        -0.474610 -0.778683  

Experiment: 2xdust
                    ARC DJF   ARC MAM   ARC JJA   ARC SON   ANT DJF   ANT MAM  \
IPSL-CM6A-LR-INCA  0.557548 -0.596719 -0.590377  0.374632 -0.292732  0.030561   
UKESM1-0-LL       -0.096941  0.280963 -0.422601  0.101109 -0.039652 -0.178474   
NorESM2-LM         0.089839  0.680931  0.548995  0.042616 -0.334020  0.565876   

                    ANT JJA   ANT SON  
IPSL-CM6A-LR-INCA -0.249523 -0.071822  
UKESM1-0-LL        0.129572  0.117032  
NorESM2-LM         1.228110  0.92

In [23]:
import pandas as pd
import numpy as np

# Suppose the below variables are already defined
# bias : shape (len(region_season_label), len(experiments)-1, len(models))
# region_season_label : list of region/season names
# experiments : list of experiments, with experiment[0] = "control"
# models : list of model names

all_data = []  # Will hold the melted DataFrame for each experiment

for idx_exp in range(len(experiments) - 1):
    exp_name = experiments[idx_exp + 1]
    print("Experiment:", exp_name)

    # bias[:, idx_exp, :].shape -> (n_region_season, n_models)
    # transpose -> (n_models, n_region_season)
    df = pd.DataFrame(
        data=bias[:, idx_exp, :].transpose(),
        index=models,
        columns=region_season_label
    )

    # Reshape df to "long" format with columns [Model, region_season_label, value]
    df_long = df.reset_index().melt(
        id_vars="index",                 # "index" column is the model
        var_name="RegionSeason",         # name for the melted columns
        value_name="Bias"               # name for the melted values
    )

    # Rename "index" -> "Model"
    df_long.rename(columns={"index": "Model"}, inplace=True)

    # Add a column for Experiment
    df_long["Experiment"] = exp_name

    # Append to the list
    all_data.append(df_long)

# Concatenate all experiments into one DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Optionally reorder columns (Experiment, Model, RegionSeason, Bias)
final_df = final_df[["Experiment", "Model", "RegionSeason", "Bias"]]

# Save to CSV
final_df.to_csv("all_bias_results.csv", index=False)

print("CSV file saved: all_bias_results.csv")


Experiment: 2xss
Experiment: 2xdust
Experiment: 2xfire
Experiment: 2xDMS
CSV file saved: all_bias_results.csv
